In [202]:
from graphviz import Digraph
import copy
import re




def insert_sequence(str1, str2, index):
    '''
    just a helper function that injects str2 into str1 just after the index given.
    used when resolving the range operator
    '''
    str1_split1 = str1[:index]
    str1_split2 = str1[index:]
    new_string = str1_split1 + str2 + str1_split2
    return new_string





def ClassesPreprocessor(infix):
    '''
    loops on all square brackets, checks for consecutive alphaneumerics, inserts | between them
    so in the case of [a-zA-Z] will be [a-z|A-Z] and in the case of [abc] will be [a|b|c] and the case [a-z] will be [a-z] nothing will change here
    THIS FUNCTION IS TO BE RUN BEFORE 'preprocessor' FUNCTION or you will face some lovely bugs :)
    '''
    alphabet = list("0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ")
    processed_infix = []
    insideClass = False
    for i,c in enumerate(infix):
        processed_infix.append(c)
        if c == '[':
            insideClass = True
            continue
        if c == ']':
            insideClass = False
        if(insideClass and c in alphabet and infix[i+1] in alphabet):
            processed_infix.append('|')
    
    return ''.join(processed_infix)

        




def preprocessor(infix):
    '''
    loop on every character c in infix :
	if c is in ['*' , '+', '?',')',']'] and if next character v is anything not in ['*' , '+', '?',')',']' , '.', '|'] :
		append ' . ' right between c and v  (right after c)

    if c is alphabet or digit and next char v is alphabet or digit or opening brackets '(' or '[' :
        append '.' right between c and v (right after c)
    '''
    specials = ['*' , '+', '?',')',']']
    others = ['.', '|']
    alphabet = list("0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ")
    
    out_string = ""
    out_index = 0

    for i, c in enumerate(infix):
        out_string = out_string + c
        out_index = out_index + 1
        if i < len(infix) - 1:
            if c in specials and not(infix[i+1] in specials or infix[i+1] in others):
                out_string = insert_sequence(out_string, '.', out_index)
                out_index = out_index + 1

            elif(c in alphabet and (infix[i+1] in alphabet or infix[i+1] in ['(', '['])):
                out_string = insert_sequence(out_string, '.', out_index)
                out_index = out_index + 1

    return out_string





def shunt(infix):
    ''' Shunting Yard algorithm, to be run after the preprocessing steps obvs '''
  
    alphabet = list("0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ")
    operators = {'*': 5,                  \
               '+': 4,                  \
                '?': 3,                 \
                '.': 2,                 \
                '|': 1}
  


    # Initializing empty pofix and stack strings
    postfix, stack = "", ""  #a stack is just a string or list :D
    i = 0

    #for every character c in input infix
    while infix != [] and i < len(infix):
        c = infix[i]
        #print(f'\nindex: {i} in infix: {infix} looking at character: {c}')
        
        #if opening bracket
        if c in ['(' , '[']:
            stack = stack + c  #push c to stack

        #if closing bracket
        elif c == ')':
            # pop until we reach the matching bracket
            while stack[-1] != '(':
                if stack == []:                    #if stack became empty, meaning the bracket is not opened aslan, then raise error
                    return False
                postfix = postfix + stack[-1]  #append to postfix
                stack = stack[:-1]  #then pop stack

            stack = stack[:-1]  # another pop to remove the open bracket in the stack, it is useless
    

        elif c == ']':
            # pop until we reach the matching bracket
            while stack[-1] != '[':
                if stack == []:                    #if stack became empty, meaning the bracket is not opened aslan
                    return False
                postfix = postfix + stack[-1]  #append to postfix
                stack = stack[:-1]  #pop stack

            stack = stack[:-1]  # another pop to remove the open bracket in the stack


        #if c is an operator
        elif c in operators:
            while stack and operators[c] <= operators.get(stack[-1], 0):      #if stack not empty and precedence(c) <= precedence(top of stack)
                postfix, stack = postfix + stack[-1], stack[:-1]                #pop and append to postfix
        
            stack = stack + c                                                 #just push c to stack



        elif c == '-':
            if(i == len(infix)):                                     #raise error in case of missing second operand bcs input is finished, ex: [A -
                return False
            first = postfix[-1]
            last = infix[i+1]
            if(not(last in alphabet)):                               #raise error in case of second operand is not in alphabet, ex: [A - ()]
                return False

            '''
            construct the new list to be injected into infix, example [a-f] we want to replace '-' by '|b|c|d|e|'
            so we want to get the starting character and the ending character, then construct the list
            then finally inject it into infix using the function insert_sequence
            '''
            processed_list = []
            for v in alphabet:
                if(alphabet.index(v) > alphabet.index(first) and alphabet.index(v) < alphabet.index(last)):
                    processed_list.append('|')
                    processed_list.append(v)
            
            processed_list.append('|')
            infix = insert_sequence(infix, ''.join(processed_list), i+1)
           



        else:   #just a normal character or digit, nothing special, just append to postfix
            postfix = postfix + c

        #print(f'postfix: {postfix}')
        i = i+1

    #finalizing step after infix is empty now, just pop all the stack and append them all to postfix
    while stack:
        if stack[-1] in ['(', '[']:                             #if we found an opened bracket but not closed when finalizing (no more inputs)
            return False
        postfix, stack = postfix + stack[-1], stack[:-1]

    #return the result
    return postfix


In [203]:
class edge:
      def __init__(self):
        self.label=None
        self.destination=None
      def __str__(self):
        return f"edge {self.label} going to {self.destination}"
  



class state:
  def __init__(self):
    self.label=None
    self.Out_Edges=[]

  def __repr__(self) -> str:
     return self.label
  def __eq__(self, other):
    return other.label == self.label
  def __ne__(self, other):
    return other.label != self.label
  def __hash__(self):
     return hash(self.label)





class nfa:

    def __init__(self, initial, acc, inner):
        self.start = initial
        self.accept = acc
        self.inner_states = inner
    def __repr__(self) ->str:
      return self.start.label




def concatNFA (stack):
   
    
    nfa2 = stack.pop()
    nfa1 = stack.pop()


    nEdge = edge()
    nEdge.label = 'ε'
    nEdge.destination = nfa2.start
    nfa1.accept.Out_Edges.append(nEdge)

    #print(nfa1)
    #print(nfa1.inner_states)

    resultnfa = nfa(nfa1.start,  nfa2.accept, nfa1.inner_states + nfa2.inner_states)
    stack.append(resultnfa)
    return resultnfa

    


def orNFA (stack, id):
  nfa1 = stack.pop()
  nfa2 = stack.pop()
  newStart = state()
  newStart.label= "S"+str(id)
  newEnd = state()
  newEnd.label = "S"+str(id+1)

  nEdge1 = edge()
  nEdge1.label = 'ε'
  nEdge1.destination = nfa1.start

  nEdge2 = edge()
  nEdge2.label = 'ε'
  nEdge2.destination = nfa2.start

  newStart.Out_Edges.append(nEdge1)
  newStart.Out_Edges.append(nEdge2)


  nEdge3 = edge()
  nEdge3.label = 'ε'
  nEdge3.destination = newEnd

  nfa1.accept.Out_Edges.append(nEdge3)

  nEdge4 = edge()
  nEdge4.label = 'ε'
  nEdge4.destination = newEnd
  nfa2.accept.Out_Edges.append(nEdge4)


  result = nfa(newStart, newEnd, [newStart,newEnd]+ nfa1.inner_states + nfa2.inner_states)
  stack.append(result)
  return result, id+2






def ZeroMoreNFA (stack, id):
  nfa1 = stack.pop()



  newStart = state()
  newStart.label = "S"+str(id)
  newEnd = state()
  newEnd.label = "S"+str(id+1)

  nEdge1 = edge()
  nEdge1.label = 'ε'
  nEdge1.destination = newStart
  nfa1.accept.Out_Edges.append(nEdge1)



  nEdge2 = edge()
  nEdge2.label = 'ε'
  nEdge2.destination=nfa1.start
  newStart.Out_Edges.append(nEdge2)

  nEdge3 = edge()
  nEdge3.label = 'ε'
  nEdge3.destination = newEnd
  nfa1.accept.Out_Edges.append(nEdge3)



  nEdge4 = edge()
  nEdge4.label = 'ε'
  nEdge4.destination = newEnd
  newStart.Out_Edges.append(nEdge4)
  
  result = nfa (newStart, newEnd, [newStart, newEnd] + nfa1.inner_states)
  stack.append(result)
  return result, id+2




def OneMoreNFA (stack, id):
   nfa1 = stack.pop()



   newStart = state()
   newStart.label = "S"+str(id)
   newEnd = state()
   newEnd.label = "S"+str(id+1)

   nEdge1 = edge()
   nEdge1.label ='ε'
   nEdge1.destination = newStart
   nfa1.accept.Out_Edges.append(nEdge1)

   nEdge2 = edge()
   nEdge2.label='ε'
   nEdge2.destination=nfa1.start
   newStart.Out_Edges.append(nEdge2)

   nEdge3 = edge()
   nEdge3.label='ε'
   nEdge3.destination = newEnd
   nfa1.accept.Out_Edges.append(nEdge3)


   result = nfa (newStart, newEnd, [newStart, newEnd] + nfa1.inner_states)
   stack.append(result)
   return result, id+2



def ZeroOneNFA (stack, id):
  nfa1 = stack.pop()

  newStart = state()
  newStart.label = "S"+str(id)
  newEnd = state()
  newEnd.label = "S"+str(id+1)

  nEdge1 = edge()
  nEdge1.label='ε'
  nEdge1.destination=nfa1.start
  newStart.Out_Edges.append(nEdge1)

  nEdge2 = edge()
  nEdge2.label='ε'
  nEdge2.destination = newEnd
  nfa1.accept.Out_Edges.append(nEdge2)

  
  nEdge2 = edge()
  nEdge2.label='ε'
  nEdge2.destination = newEnd
  newStart.Out_Edges.append(nEdge2)



  result = nfa (newStart, newEnd, [newStart, newEnd] + nfa1.inner_states)
  stack.append(result)
  return result, id+2




def ConstructNFA (c:str, id:int, stack):
    start = state()
    accept = state()
    start.label = "S"+str(id)
    accept.label = "S"+str(id+1)

    nEdge = edge()
    nEdge.label = c
    nEdge.destination = accept
    start.Out_Edges.append(nEdge)
    


    
    result_nfa = nfa(start, accept, [start,accept])
    stack.append(result_nfa)
    #print(result_nfa.inner_states)

    return result_nfa, id+2



In [204]:
class DFA_Edge:
    def __init__(self):
      self.label = None
      self.destination = None           #destination here points to a SuperState not a state


class SuperState:
    def __init__(self):
        self.isStart = None          #boolean
        self.isEnd = None            #boolean
        self.Out_Edges = []        #list of edges
        self.SubStates = set()        #list of states
        self.label = None

    def __eq__(self, other):
      set_of_src_labels = set()
      set_of_dst_labels = set()
      for s in self.SubStates:
        set_of_src_labels.add(s.label)
      for s in other.SubStates:
        set_of_dst_labels.add(s.label)
      return set_of_src_labels == set_of_dst_labels

    def __ne__(self, other):
      set_of_src_labels = set()
      set_of_dst_labels = set()
      for s in self.SubStates:
        set_of_src_labels.add(s.label)
      for s in other.SubStates:
        set_of_dst_labels.add(s.label)
      return set_of_src_labels != set_of_dst_labels

    def __str__(self):
      if(self.SubStates):
        stringy = ""
        for sub in self.SubStates:
          stringy = stringy + str(sub) + ","
        return "SuperState " +stringy
      else:
        return "Empty SuperState"

    def __repr__(self):
      if(self.SubStates):
        stringy = ""
        for sub in self.SubStates:
          stringy = stringy + str(sub) + ","
        return stringy
      else:
        return "Empty SuperState"

    def __hash__(self):
      if(self.SubStates):
        stringy = " "
        for sub in self.SubStates:
          stringy = stringy + str(sub) + ","
        return hash("SuperState" +stringy)
      else:
        return hash(None)


class DFA:
    def __init__(self):
        self.StartSTT = None          #StartSuperState
        self.EndSTT = None            #ending SuperStates
        self.SuperSS = set()           #list of SuperStates
    def __eq__(self, other):
      return self.StartSTT == other.StartSTT and self.SuperSS == self.SuperSS
    def __ne__(self, other):
      return self.StartSTT != other.StartSTT or self.SuperSS != self.SuperSS
    def empt(self):
      self.StartSTT = SuperState()          #StartSuperState
      self.EndSTT = False            #ending SuperStates
      self.SuperSS = set()           #list of SuperStates

def EpsClosure(stat:state):
    epsStates = set()
    epsStates.add(stat)
    for edg in stat.Out_Edges:
        #print(f"state {stat} -> "+str(edg))
        if edg.label == 'ε':
            epsStates.add(edg.destination)
    #print(epsStates)
    return epsStates


def CharClosure(stat:state, C):
    closeStates = set()
    #closeStates.add(stat)
    for edg in stat.Out_Edges:
        if edg.label == C:
            closeStates.add(edg.destination)
    
    return closeStates


def getLabelsfromSuperState(SSTT:SuperState):
  list_of_labels=[]
  for st in SSTT.SubStates:
    list_of_labels.append(st.label)
  return list_of_labels



def followChar (stat:state, C):
    reachable_states = set()
    unchecked_states = set()
    unchecked_states.add(stat)
    checked_states = set()              #leave it as it is until we are sure that there will be no eps cycles
    while(unchecked_states):
        to_be_checked = unchecked_states.copy()
        for S in to_be_checked:
            reachable_states.update(CharClosure(S, C))
            unchecked_states.update(CharClosure(S, C))
            unchecked_states.remove(S)
    return reachable_states if reachable_states else False


def followEps(stat:state):
    reachable_states = set()
    reachable_states.add(stat)
    unchecked_states = set()
    unchecked_states.add(stat)
    checked_states = set()              #leave it as it is until we are sure that there will be no eps cycles
    while(unchecked_states != set()):
        to_be_checked = unchecked_states.copy()
        for S in to_be_checked:
            reachable_states.update(EpsClosure(S))
            unchecked_states.update(EpsClosure(S))
            unchecked_states.remove(S)
    #print(reachable_states)
    return reachable_states if reachable_states else False



def construct_DFA_Starting_Node(NFA:nfa):
  initial_superSet = SuperState()
  initial_superSet.isStart = True
  initial_superSet.label = "S0"
  if(not followEps(NFA.start)):
    print(f"starting state has no Epsilon transmissions")
  initial_inner_states = followEps(NFA.start)
  initial_superSet.SubStates.update(initial_inner_states)
  initial_superSet.Out_Edges = []

  startingDFA = DFA()
  startingDFA.StartSTT = initial_superSet
  startingDFA.EndSTT = []
  startingDFA.SuperSS.add(initial_superSet)

  return startingDFA                               #returns the initial DFA



def make_new_superSets(NFA:nfa, SSTT:SuperState, inputChar):
  ListOfSubStates = set()
  if(SSTT.SubStates == set()):
    print(f"input SuperState does not exist : {SSTT}")
    pass
  for stt in SSTT.SubStates:
    if(not CharClosure(stt, inputChar)):
      #print(f"no more transfers from the given input {inputChar} from the SuperState {SSTT} and sub state {stt}")
      pass
    ListOfSubStates.update(CharClosure(stt, inputChar))
    for st in ListOfSubStates.copy():
      ListOfSubStates.update(followEps(st))
  return ListOfSubStates



def getSuperState_from_list_of_states (dfa:DFA, List_of_states):
  for SSTT in dfa.SuperSS:
    if(List_of_states == SSTT.SubStates):
      return SSTT
  return False



def Expand_DFA_based_on_input(dfa:DFA, SSTT:SuperState, nefa:nfa, inputChar, id, SSTT_to_be_checked):
  SuperStatesToBeChecked = set()
  SuperStatesToBeChecked.update(dfa.SuperSS)
  #for SSTT in dfa.SuperSS:
  if (SSTT):
    #SSTT = SuperStatesToBeChecked.pop()
    #print(SSTT)
    print(f"will now check for {SSTT} for input {inputChar}")
    ListOfSubStates = make_new_superSets(nefa, SSTT, inputChar)                  ##########################
    #print(f"\n\n{ListOfSubStates}\n\n")
    if(not ListOfSubStates):
      return dfa, id
    existingSubStates = []
    for ss in dfa.SuperSS:
      existingSubStates.append(set(ss.SubStates))
    #print(f"List Of SubStates : {ListOfSubStates}")
    print(f"List Of SubStates : {existingSubStates}")
    if(ListOfSubStates in existingSubStates):
      #if the SuperState already exists, either make a new edge or break and continue to the next SuperState



      #if an edge already exists, then continue
      for edg in SSTT.Out_Edges:
        if(edg.destination == getSuperState_from_list_of_states(dfa, ListOfSubStates) and edg.label == inputChar): #equality operator overload of superstate
          continue

        #else
        #make an edge to this state and continue
        else:
          #print(f"will construct edge {inputChar} from {SSTT} to {getSuperState_from_list_of_states(dfa, ListOfSubStates)}")
          newEdge1 = DFA_Edge()
          newEdge1.label = inputChar
          if(not getSuperState_from_list_of_states(dfa, ListOfSubStates)):
            print(f'no super state matches the given list of states : {ListOfSubStates}')
            pass
          newEdge1.destination = getSuperState_from_list_of_states(dfa, ListOfSubStates)
          SSTT.Out_Edges.append(newEdge1)
      
      if (SSTT.Out_Edges == []):
        #print(f"will construct edge {inputChar} from {SSTT} to {getSuperState_from_list_of_states(dfa, ListOfSubStates)}")
        newEdge2 = DFA_Edge()
        newEdge2.label = inputChar
        if(not getSuperState_from_list_of_states(dfa, ListOfSubStates)):
          print(f'no super state matches the given list of states : {ListOfSubStates}')
          pass
        newEdge2.destination = getSuperState_from_list_of_states(dfa, ListOfSubStates)
        SSTT.Out_Edges.append(newEdge2)

      

    else:
      #this superstate does not exist, make a new one
      newSuperState = SuperState()
      newSuperState.isStart = False
      newSuperState.SubStates.update(ListOfSubStates)
      newSuperState.label = "S"+ str(id)
      id = id + 1
      newEdge = DFA_Edge()
      newEdge.label = inputChar
      newEdge.destination = newSuperState
      SSTT.Out_Edges.append(newEdge)
      dfa.SuperSS.add(newSuperState)
      SSTT_to_be_checked.append(newSuperState)
      #SuperStatesToBeChecked.add(newSuperState)
    

  return dfa, id



def getInputCharacters(postfix):
  alphabet = list("0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ")
  inputChars = set()
  for c in postfix:
    if c in alphabet:
      inputChars.add(c)
  return list(inputChars)



def CompletedDFA (NFA:nfa, postfix):
  inputChars = getInputCharacters(postfix)
  #print(inputChars)
  initDFA = construct_DFA_Starting_Node(NFA)
  tempDFA1 = DFA()
  tempDFA1.empt()
  itera = 0

  id = 1

  countdown = 3

    #if (tempDFA1 != initDFA):
      #print(itera)
  tempDFA1 = copy.copy(initDFA)
  

  SSTT_to_be_checked = []
  SSTT_to_be_checked = list(initDFA.SuperSS)
  i = 0
  while SSTT_to_be_checked != []:
    SSTT = SSTT_to_be_checked.pop()
    for inputc in inputChars:
      initDFA, id = Expand_DFA_based_on_input(initDFA, SSTT, NFA, inputc, id, SSTT_to_be_checked)
    i = i + 1
    
        
    itera = itera + 1

  for SSTT in initDFA.SuperSS:
    if NFA.accept.label in getLabelsfromSuperState(SSTT):
      SSTT.isEnd = True
    else:
      SSTT.isEnd = False
  return initDFA

    

'\ndef CleanseDFA (dfa:DFA):\n  cleanDFA = DFA()\n  cleanDFA.empt()\n  for SSTT in dfa.SuperSS:\n    newSuperState = SuperState()\n    newSuperState.isStart = SSTT.isStart\n    newSuperState.isEnd = SSTT.isEnd\n    newSuperStates.SubStates = \n      #newSuperState.label = "S"+ str(id)\n      id = id + 1\n      newEdge = DFA_Edge()\n      newEdge.label = inputChar\n      newEdge.destination = newSuperState\n      SSTT.Out_Edges.append(newEdge)\n      dfa.SuperSS.add(newSuperState)\n      SSTT_to_be_checked.append(newSuperState)\n    cleanDFA.SuperSS.append(SSTT)\n'

In [205]:

def vizualize_NFA(resultNFA):

  #gra = Digraph(graph_attr={'landscape':'True'})
  gra = Digraph(graph_attr={'rankdir':'LR'})


  #construct nodes first
  for stat in resultNFA.inner_states:
      if(stat.label == resultNFA.start.label):
        gra.node("", _attributes={'shape' : 'none'})
        gra.edge("", stat.label)
      if(stat.label == resultNFA.accept.label):
        gra.node(stat.label, _attributes={'peripheries' : '2'})
      else:
        gra.node(stat.label)

  #for each node, construct edges
  for stat in resultNFA.inner_states:
      for edg in stat.Out_Edges:
          gra.edge(stat.label, edg.destination.label, label=edg.label)

  return gra.source


In [206]:
def Vizualize_DFA (dfa:DFA):
  gra = Digraph(graph_attr={'rankdir':'LR'})
  
  id = 0
  for SSTT in dfa.SuperSS:
    if(SSTT.isStart):
      gra.node("", _attributes={'shape' : 'none'})
      gra.edge("", repr(SSTT))
    if(SSTT.isEnd):
      #gra.node(SSTT.label, _attributes={'peripheries' : '2'})
      gra.node(repr(SSTT), _attributes={'peripheries' : '2'})
    else:
      #gra.node(SSTT.label)
      gra.node(repr(SSTT))
    id = id + 1
    

  for SSTT in dfa.SuperSS:
    labelsOfSource = ""
    for stt in SSTT.SubStates:
      labelsOfSource = labelsOfSource + stt.label + " , "

    
    for edg in SSTT.Out_Edges:
      #labelsOfDest = ""
      #for stt in edg.destination.SubStates:
      #  labelsOfDest = labelsOfDest + stt.label + " , "
      #print(labelsOfSource)
      gra.edge(repr(SSTT) , repr(edg.destination), label=edg.label)

  return gra.source






def Vizualize_cleansed_DFA (dfa:DFA):
  gra = Digraph(graph_attr={'rankdir':'LR'})
  
  id = 0
  for SSTT in dfa.SuperSS:
    if(SSTT.isStart):
      gra.node("", _attributes={'shape' : 'none'})
      gra.edge("", SSTT.label)
    if(SSTT.isEnd):
      #gra.node(SSTT.label, _attributes={'peripheries' : '2'})
      gra.node(SSTT.label, _attributes={'peripheries' : '2'})
    else:
      #gra.node(SSTT.label)
      gra.node(SSTT.label)
    id = id + 1
    

  for SSTT in dfa.SuperSS:
    labelsOfSource = ""
    for stt in SSTT.SubStates:
      labelsOfSource = labelsOfSource + stt.label + " , "

    
    for edg in SSTT.Out_Edges:
      #labelsOfDest = ""
      #for stt in edg.destination.SubStates:
      #  labelsOfDest = labelsOfDest + stt.label + " , "
      #print(labelsOfSource)
      gra.edge(SSTT.label , edg.destination.label, label=edg.label)

  return gra.source

In [207]:

def PostFix_To_NFA(postfix):
  alphabet = list("0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ")


  stack = []
  id = 1

  for c in postfix:
    if c in alphabet:
      _, id = ConstructNFA(c, id, stack)
    elif c == '*':
      _, id = ZeroMoreNFA(stack, id)
    elif c == '+':
      _, id = OneMoreNFA(stack, id)
    elif c == '?':
      _, id = ZeroOneNFA(stack, id)
    elif c == '.':
      concatNFA(stack)
    elif c == '|':
      _, id = orNFA(stack, id)
    else:
      print(f"unknown operator {c}")

  result = stack.pop()
  return result, vizualize_NFA(result)

In [208]:


inRegex = "(a|b)*a[ab]?"
infix = re.sub(" ", "", inRegex)


classes_handled = ClassesPreprocessor(infix)
preprocessed = preprocessor(classes_handled)
postfix = shunt(preprocessed)
print(f"\n\nfinal:          {postfix}")




nefa, nfa_GraphSrc = PostFix_To_NFA(postfix)
print(vizualize_NFA(nefa))



final = CompletedDFA(nefa, postfix)


src2 = Vizualize_DFA(final)
print(src2)





final:          ab|*a.ab|?.
digraph {
	graph [rankdir=LR]
	"" [shape=none]
	"" -> S7
	S7
	S8
	S5
	S6
	S3
	S4
	S1
	S2
	S9
	S10
	S17
	S18 [peripheries=2]
	S15
	S16
	S13
	S14
	S11
	S12
	S7 -> S5 [label="ε"]
	S7 -> S8 [label="ε"]
	S8 -> S9 [label="ε"]
	S5 -> S3 [label="ε"]
	S5 -> S1 [label="ε"]
	S6 -> S7 [label="ε"]
	S6 -> S8 [label="ε"]
	S3 -> S4 [label=b]
	S4 -> S6 [label="ε"]
	S1 -> S2 [label=a]
	S2 -> S6 [label="ε"]
	S9 -> S10 [label=a]
	S10 -> S17 [label="ε"]
	S17 -> S15 [label="ε"]
	S17 -> S18 [label="ε"]
	S15 -> S13 [label="ε"]
	S15 -> S11 [label="ε"]
	S16 -> S18 [label="ε"]
	S13 -> S14 [label=b]
	S14 -> S16 [label="ε"]
	S11 -> S12 [label=a]
	S12 -> S16 [label="ε"]
}

will now check for SuperState S8,S5,S7,S3,S1,S9, for input a
List Of SubStates : [{S8, S5, S7, S3, S1, S9}]
will now check for SuperState S8,S5,S7,S3,S1,S9, for input b
List Of SubStates : [{S8, S5, S7, S3, S1, S9}, {S8, S5, S3, S7, S17, S18, S11, S2, S1, S10, S6, S13, S15, S9}]
will now check for SuperState S8,S5,S7

In [209]:


src3 = Vizualize_cleansed_DFA(final)
print(src3)

digraph {
	graph [rankdir=LR]
	"" [shape=none]
	"" -> S0
	S0
	S4 [peripheries=2]
	S1 [peripheries=2]
	S3 [peripheries=2]
	S2
	S0 -> S1 [label=a]
	S0 -> S2 [label=b]
	S4 -> S1 [label=a]
	S4 -> S2 [label=b]
	S1 -> S3 [label=a]
	S1 -> S4 [label=b]
	S3 -> S3 [label=a]
	S3 -> S4 [label=b]
	S2 -> S1 [label=a]
	S2 -> S2 [label=b]
}

